In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
import random

from keras import models, layers, optimizers, callbacks
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.densenet import DenseNet121

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATASET_DIR = '/kaggle/input/gtsrb-german-traffic-sign'

TRAIN_DIR = DATASET_DIR + '/Train'
TEST_DIR = DATASET_DIR + '/Test'
MAX_EPOCHS = 100
BATCH_SIZE = 128
LR = 1e-3

In [ ]:
train_x = []
train_y = []

for i in range(43):
    print(f'Reading class: {i}')
    CLASS_DIR = f'{TRAIN_DIR}/{i}/'
    
    label = np.zeros((43, ))
    label[i] = 1
    
    for file in random.choices(os.listdir(CLASS_DIR), k=1000):
        
        image = Image.open(CLASS_DIR + file)
        image = image.resize((64, 64))
        image = np.asarray(image)
        
        train_x.append(image)
        train_y.append(label)
        
@tf.function
def image_augment(x, y): 
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
        
    if p_rotate > .8:
        x = tf.image.rot90(x, k=3) 
    elif p_rotate > .6:
        x = tf.image.rot90(x, k=2) 
    elif p_rotate > .4:
        x = tf.image.rot90(x, k=1)
        
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    
    if p_spatial > .75:
        x = tf.image.transpose(x)
        
    return x, y
    
train_x = np.asarray(train_x, dtype='float32') / 255
train_y = np.asarray(train_y, dtype='float32')

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))

train_dataset = train_dataset.map(image_augment)

train_dataset = train_dataset.shuffle(2048)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_x = []
test_y = []
test_df = pd.read_csv(DATASET_DIR + '/Test.csv')

for file in random.choices(os.listdir(TEST_DIR), k=1000):
    if not '.png' in file:
        continue
    
    class_id = test_df[test_df['Path'] == ('Test/' + file)]['ClassId']
    
    image = Image.open(TEST_DIR + '/' + file)
    image = image.resize((64, 64))
    image = np.asarray(image)
    
    label = np.zeros(43)
    label[class_id] = 1

    test_x.append(image)
    test_y.append(label)

    
test_x = np.asarray(test_x, dtype='float32') / 255
test_y = np.asarray(test_y, dtype='float32')


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

test_dataset = test_dataset.map(image_augment)

test_dataset = test_dataset.shuffle(2048)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
def scheduler(epoch):
    if epoch < 4:
        return 0.0005
    elif epoch < 8:
        return 0.0002
    elif epoch < 12:
        return 0.0001
    elif epoch < 16:
        return 0.00005
    elif epoch < 20:
        return 0.00002
    else:
        return 0.00001
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = True)

In [ ]:
base_mobile_model = MobileNet(
    input_shape=(64, 64, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
base_mobile_model.trianable = True


mobile_model = models.Sequential([
    base_mobile_model,
    
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(43, activation='softmax')
])

mobile_model.compile(optimizer=optimizers.rmsprop_v2.RMSprop(learning_rate=0.0005), loss='categorical_crossentropy', metrics='accuracy')
mobile_model.summary()

history = mobile_model.fit(train_dataset, validation_data=(test_dataset), epochs=MAX_EPOCHS, callbacks=[
    callbacks.EarlyStopping('val_accuracy', patience=5),
    lr_callback
])

In [ ]:
base_dense_model = DenseNet121(
    input_shape=(64, 64, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
base_dense_model.trianable = True


dense_model = models.Sequential([
    base_dense_model,
    
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(43, activation='softmax')
])

dense_model.compile(optimizer=optimizers.rmsprop_v2.RMSprop(learning_rate=0.0005), loss='categorical_crossentropy', metrics='accuracy')
dense_model.summary()

history = dense_model.fit(train_dataset, validation_data=(test_dataset), epochs=MAX_EPOCHS, callbacks=[
    callbacks.EarlyStopping('val_accuracy', patience=5),
    lr_callback
])

In [ ]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in test_dataset:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(np.argmax(label_batch, axis=-1))
   # compute predictions
   preds_mobile = mobile_model.predict(image_batch)
   preds_dense = dense_model.predict(image_batch)
   preds = (preds_mobile + preds_dense) / 2
   # append predicted labels
   y_pred.append(np.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [ ]:
def display_confusion_matrix(cmat):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Blues')
    ax.set_xticks(range(43))
    ax.set_xticklabels(range(43), fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(43))
    ax.set_yticklabels(range(43), fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""

    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_true=correct_labels, y_pred=predicted_labels)
display_confusion_matrix(confusion_matrix)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=correct_labels, y_pred=predicted_labels)